In [9]:
import sys
import re
import pandas as pd
from sklearn.utils import resample
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.svm import SVC
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, f1_score,precision_score,classification_report,log_loss


In [13]:
stemmer = SnowballStemmer('english')    
words =  stopwords.words('english')

In [14]:
class Hate(object):
    def cleanTrain(self):
        #opening tweet filese
        data=pd.read_csv('normal-binary-hate.csv',encoding="ISO-8859-1")
      

        print('rows: '%data.columns, data.shape,len(data))

        #cleaning the tweets
        def  clean_text(df, text_field):
            corpus=[]
            df=df.dropna()
            df = df.reset_index(drop=True)
            df[text_field] = df[text_field].str.lower()
            for i in range(0,len(df.copy())):
                review=re.sub('@[^\s]+','', df[text_field][i])
                review=re.sub(r'https?:\/\/\S+', '',review)
                review=re.sub(r'^RT[\s]+', '',review)
                review=re.sub(r'#', '',review)
                review=re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '',review)
                review=re.sub(r'{link}', '',review)
                review=re.sub(r'&[a-z]+;', '',review)
                review=re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '',review)
                review=review.split()
                review=[stemmer.stem(word) for word in review if not word in words]
                review=" ".join(review)
                corpus.append(review)
            return corpus



        
            


        data_clean = clean_text(data, "tweet")
        X=data_clean
        print(f'cleaned X variable is a {type(X)}')
        Y =data['label'].values
      

    
        self.tfidf=TfidfVectorizer(max_features=5000)
        X=self.tfidf.fit_transform(X)
        #splitting to training and testing data
        X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state = 0)

    
#         clf=LinearSVC()
#         param_grid = {'C': [0.1, 1, 10, 100, 1000], 
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#               'kernel': ['rbf','linear','sigmoid','poly']} 
  
#         grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
#         print(rnd_search_cv.best_estimator_)
  
        # fitting the model for grid search
        clf=SVC(C=1,gamma=1, kernel='rbf')
        self.model = clf.fit(X_train, y_train)
        y_predict = self.model.predict(X_test)
        filename = 'finalized_model.sav'
        pickle.dump(self.model, open(filename, 'wb'))
        
        score=precision_score(y_test, y_predict)
        acc_score=accuracy_score(y_test,y_predict)
        print(f'Precision score is{score}')
        print(f'Accuracy score is{acc_score}')
        print(classification_report(y_test,y_predict))
        return



    def interface(self,status):
        
        #cleaning the post
        text_corpus=[]
        text =re.sub('@[^\s]+','', status.text)
        text=re.sub(r'https?:\/\/\S+', '',text)
        text=re.sub(r'^RT[\s]+', '',text)
        text=re.sub(r'#', '',text)
        text=re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '',text)
        text=re.sub(r'{link}', '',text)
        text=re.sub(r'&[a-z]+;', '',text)
        text=re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '',text)
        text=text.lower()
        tfidf=self.tfidf
        vec=tfidf.transform([text])
        print(f"Value of vec is {type(vec)}")

        my_val=(self.model.predict(vec)> 0.5).astype("int32")
      
        print(f'my_val:{my_val}')
        useful = {'weight':my_val}
    
        return useful

In [15]:
k= Hate()
k.cleanTrain()
#         m=k.interface(status)

rows:  (29946, 3) 29946
cleaned X variable is a <class 'list'>
Precision score is0.9730517115804806
Accuracy score is0.9278797996661102
Log loss is2.4909582901362555
              precision    recall  f1-score   support

           0       0.83      0.93      0.88      1661
           1       0.97      0.93      0.95      4329

    accuracy                           0.93      5990
   macro avg       0.90      0.93      0.91      5990
weighted avg       0.93      0.93      0.93      5990

